In [11]:
import pandas as pd

data_path = "data/flyvis_data"
cell_type_df = pd.read_csv(f'{data_path}/flyvis_cell_type_connectivity.csv')
outgoing_pert_path = f"{data_path}/outgoing_edge_pert_moving_edge_results"
pair_wise_pert_path = f"{data_path}/pairwise_edge_pert_results"
origianl_network_res = pd.read_csv(f"{data_path}/mv_edge_original_network.csv")

In [12]:
origianl_network_res

,cell_type,pd_intensity1,pd_intensity0,dsi_intensity1,dsi_intensity0,dsi_correlation,tc_correlation_intensity1,tc_correlation_intensity0
0,T4a,3.080085,-0.133295,0.429780,0.079998,0.887835,0.937000,-0.534884
1,T4b,-0.161833,-2.677298,0.537967,0.010725,0.887835,0.719449,0.445971
2,T4c,0.981502,-2.334430,0.627362,0.092382,0.887835,0.537218,-0.164373
3,T4d,-1.506840,1.538510,0.098756,0.246006,0.887835,0.856611,-0.786793
4,T5a,0.351713,-3.042270,0.325475,0.778324,0.887835,-0.411445,0.841254
5,T5b,3.000525,-0.189463,0.172208,0.748803,0.887835,-0.308164,0.903209
6,T5c,-0.668891,1.822082,0.000583,0.579232,0.887835,-0.312159,0.949565
7,T5d,1.365778,-1.946797,0.034477,0.529629,0.887835,-0.442423,0.901005


### outgoing analysis

In [ ]:
for src_cell_type in cell_type_df.source_type.unique():
    print(f"Processing source cell type: {src_cell_type}")
    outgoing_df = pd.read_csv(f"{outgoing_pert_path}/{src_cell_type}_outgoing.csv")
    break

Processing source cell type: Am


In [9]:
outgoing_df

,cell_type,pd_intensity1,pd_intensity0,dsi_intensity1,dsi_intensity0,dsi_correlation,tc_correlation_intensity1,tc_correlation_intensity0
0,T4a,3.080085,-0.133295,0.429780,0.079998,0.887835,0.937000,-0.534884
1,T4b,-0.161833,-2.677296,0.537967,0.010725,0.887835,0.719449,0.445971
2,T4c,0.981501,-2.334429,0.627361,0.092382,0.887835,0.537218,-0.164373
3,T4d,-1.506840,1.538510,0.098756,0.246006,0.887835,0.856611,-0.786793
4,T5a,0.351713,-3.042270,0.325475,0.778324,0.887835,-0.411445,0.841254
5,T5b,3.000524,-0.189463,0.172208,0.748803,0.887835,-0.308164,0.903209
6,T5c,-0.668909,1.822082,0.000583,0.579232,0.887835,-0.312165,0.949565
7,T5d,1.365778,-1.946797,0.034477,0.529629,0.887835,-0.442423,0.901005


### pair - wise analysis